Source: [training-data-analyst, stable-link (not-master)](https://github.com/GoogleCloudPlatform/training-data-analyst/blob/f3b838007a0e1fb3cc7880ad0343f46678926f36/courses/machine_learning/deepdive/04_advanced_preprocessing/a_dataflow.ipynb) 

# Data Preprocessing for Machine Learning

**Learning Objectives**
* Understand the different approaches for data preprocessing in developing ML models
* Use Dataflow to perform data preprocessing steps

## Introduction

In the previous notebook we achieved an RMSE of **3.85**. Let's see if we can improve upon that by creating a data preprocessing pipeline in Cloud Dataflow.

Preprocessing data for a machine learning model involves both data engineering and feature engineering. During data engineering, we convert raw data into prepared data which is necessary for the model. Feature engineering then takes that prepared data and creates the features expected by the model. We have already seen various ways we can engineer new features for a machine learning model and where those steps take place. We also have flexibility as to where data preprocessing steps can take place; for example, BigQuery, Cloud Dataflow and Tensorflow. In this lab, we'll explore different data preprocessing strategies and see how they can be accomplished with Cloud Dataflow.

One perspective in which to categorize different types of data preprocessing operations is in terms of the granularity of the operation. Here, we will consider the following three types of operations:
1. Instance-level transformations
2. Full-pass transformations
3. Time-windowed aggregations

Cloud Dataflow can perform each of these types of operations and is particularly useful when performing computationally expensive operations as it is an autoscaling service for batch and streaming data processing pipelines. We'll say a few words about each of these below. For more information, have a look at this article about [data preprocessing for machine learning from Google Cloud](https://cloud.google.com/solutions/machine-learning/data-preprocessing-for-ml-with-tf-transform-pt1).

**1. Instance-level transformations**
These are transformations which take place during training and prediction, looking only at values from a single data point. For example, they might include clipping the value of a feature, polynomially expand a feature, multiply two features, or compare two features to create a Boolean flag.

It is necessary to apply the same transformations at training time and at prediction time. Failure to do this results in training/serving skew and will negatively affect the performance of the model.

**2. Full-pass transformations**
These transformations occur during training, but occur as instance-level operations during prediction. That is, during training you must analyze the entirety of the training data to compute quantities such as maximum, minimum, mean or variance while at prediction time you need only use those values to rescale or normalize a single data point. 

A good example to keep in mind is standard scaling (z-score normalization) of features for training. You need to compute the mean and standard deviation of that feature across the whole training data set, thus it is called a full-pass transformation. At prediction time you use those previously computed values to appropriately normalize the new data point. Failure to do so results in training/serving skew.

**3. Time-windowed aggregations**
These types of transformations occur during training and at prediction time. They involve creating a feature by summarizing real-time values by aggregating over some temporal window clause. For example, if we wanted our model to estimate the taxi trip time based on the traffic metrics for the route in the last 5 minutes, in the last 10 minutes or the last 30 minutes we would want to create a time-window to aggreagate these values. 

At prediction time these aggregations have to be computed in real-time from a data stream.

### Set environment variables and load necessary libraries

Apache Beam only works in Python 2 at the moment, so switch to the Python 2 kernel in the upper right hand side. Then execute the following cells to install the necessary libraries if they have not been installed already.

Check that your jupyter-server is running in the correct environement:
- I had problem if it was not one with Python 2.7, even when the kernel was selected correctly

In [ ]:
!conda info -e

In [ ]:
import tensorflow as tf
import apache_beam as beam
import shutil
import os
print("TF-Version: {}".format(tf.__version__))
import sys
print("Python-Version: {}".format(sys.version))

Next, set the environment variables related to your GCP Project.

In [ ]:
from utils import chdir_
pwd = chdir_()

import yaml
with open('config.yaml', 'r') as f:
    #cfg = yaml.load(f, Loader=yaml.BaseLoader)
    cfg = yaml.safe_load(f)

In [ ]:
PROJECT = cfg["project-id"]  # Replace with your PROJECT
BUCKET = cfg["bucket"]  # Replace with your BUCKET
REGION = cfg["region"]           # Choose an available region for Cloud MLE
TFVERSION = str(cfg["tf-version"])                # TF version for CMLE to use

In [ ]:
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION
os.environ["TFVERSION"] = TFVERSION 

## ensure we"re using python2 env
os.environ["CLOUDSDK_PYTHON"] = "python2"

In [ ]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

## ensure we predict locally with our current Python environment
gcloud config set ml_engine/local_python `which python`

## Create data preprocessing job with Cloud Dataflow

The following code reads from BigQuery and saves the data as-is on Google Cloud Storage. We could also do additional preprocessing and cleanup inside Dataflow. Note that, in this case we'd have to remember to repeat that prepreprocessing at prediction time to avoid training/serving skew. In general, it is better to use tf.transform which will do this book-keeping for you, or to do preprocessing within your TensorFlow model. We will look at how tf.transform works in another notebook. For now, we are simply moving data from BigQuery to CSV using Dataflow.

It's worth noting that while we could read from [BQ directly from TensorFlow](https://www.tensorflow.org/api_docs/python/tf/contrib/cloud/BigQueryReader), it is quite convenient to export to CSV and do the training off CSV. We can do this at scale with Cloud Dataflow. Furthermore, because we are running this on the cloud, you should go to the [GCP Console](https://console.cloud.google.com/dataflow) to view the status of the job. It will take several minutes for the preprocessing job to launch.

### Define our query and pipeline functions

To start we'll copy over the `create_query` function we created in the `01_bigquery/c_extract_and_benchmark` notebook. 

In [ ]:
def create_query(phase, sample_size):
    basequery = """
    SELECT *
    FROM
        `{project}.test.DATA`
    WHERE
        MOD(ID, EVERY_N) = 1
    """.format(project=PROJECT)

    if phase == 'TRAIN':
        subsample = """
        AND MOD(ID, EVERY_N * 100) >= (EVERY_N * 0)
        AND MOD(ID, EVERY_N * 100) <  (EVERY_N * 85)
        """
    elif phase == 'TEST':
        subsample = """
        AND MOD(ID, EVERY_N * 100) >= (EVERY_N * 85)
        AND MOD(ID, EVERY_N * 100) <  (EVERY_N * 100)
        """

    query = basequery + subsample
    return query.replace("EVERY_N", sample_size)

Then, we'll write the csv we create to a Cloud Storage bucket. So, we'll look to see that the location is empty, and if not clear out its contents so that it is.

In [ ]:
%env BUCKET_FOLDER gs://$BUCKET/mnist/bq/

In [ ]:
%%bash
echo ${BUCKET_FOLDER}
gsutil -m rm -rf ${BUCKET_FOLDER}

In [ ]:
!gsutil -m rm -rf ${BUCKET_FOLDER}

In [ ]:
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT)

sql = """
    SELECT *
    FROM `{project}.test.DATA`
    LIMIT 15
""".format(project=PROJECT)
df = client.query(sql).to_dataframe()
df.head()

In [ ]:
CSV_COLUMNS = list(df.columns)

Next, we'll create a function and pipeline for preprocessing the data. First, we'll define a `to_csv` function which takes a row dictionary (a dictionary created from a BigQuery reader representing each row of a dataset) and returns a comma separated string for each record

In [ ]:
def make_to_csv(columns):
    def to_csv(rowdict):
        """
        Arguments:
            -rowdict: Dictionary. The beam bigquery reader returns a PCollection in
            which each row is represented as a python dictionary
        Returns:
            -rowstring: a comma separated string representation of the record
        """
        days = ["null", "Sun", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat"] # unnecessary?
        #CSV_COLUMNS = "fare_amount,dayofweek,hourofday,pickuplon,pickuplat,dropofflon,dropofflat".split(',')
        #CSV_COLUMNS = ['ID', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6', 'feat_7', 'feat_8', 'feat_9', 'feat_10', 'feat_11', 'feat_12', 'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18', 'feat_19', 'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24', 'feat_25', 'feat_26', 'feat_27', 'feat_28', 'feat_29', 'feat_30', 'feat_31', 'feat_32', 'feat_33', 'feat_34', 'feat_35', 'feat_36', 'feat_37', 'feat_38', 'feat_39', 'feat_40', 'feat_41', 'feat_42', 'feat_43', 'feat_44', 'feat_45', 'feat_46', 'feat_47', 'feat_48', 'feat_49', 'feat_50', 'feat_51', 'feat_52', 'feat_53', 'feat_54', 'feat_55', 'feat_56', 'feat_57', 'feat_58', 'feat_59', 'feat_60', 'feat_61', 'feat_62', 'feat_63', 'feat_64', 'feat_65', 'feat_66', 'feat_67', 'feat_68', 'feat_69', 'feat_70', 'feat_71', 'feat_72', 'feat_73', 'feat_74', 'feat_75', 'feat_76', 'feat_77', 'feat_78', 'feat_79', 'feat_80', 'feat_81', 'feat_82', 'feat_83', 'feat_84', 'feat_85', 'feat_86', 'feat_87', 'feat_88', 'feat_89', 'feat_90', 'feat_91', 'feat_92', 'feat_93', 'feat_94', 'feat_95', 'feat_96', 'feat_97', 'feat_98', 'feat_99', 'feat_100', 'feat_101', 'feat_102', 'feat_103', 'feat_104', 'feat_105', 'feat_106', 'feat_107', 'feat_108', 'feat_109', 'feat_110', 'feat_111', 'feat_112', 'feat_113', 'feat_114', 'feat_115', 'feat_116', 'feat_117', 'feat_118', 'feat_119', 'feat_120', 'feat_121', 'feat_122', 'feat_123', 'feat_124', 'feat_125', 'feat_126', 'feat_127', 'feat_128', 'feat_129', 'feat_130', 'feat_131', 'feat_132', 'feat_133', 'feat_134', 'feat_135', 'feat_136', 'feat_137', 'feat_138', 'feat_139', 'feat_140', 'feat_141', 'feat_142', 'feat_143', 'feat_144', 'feat_145', 'feat_146', 'feat_147', 'feat_148', 'feat_149', 'feat_150', 'feat_151', 'feat_152', 'feat_153', 'feat_154', 'feat_155', 'feat_156', 'feat_157', 'feat_158', 'feat_159', 'feat_160', 'feat_161', 'feat_162', 'feat_163', 'feat_164', 'feat_165', 'feat_166', 'feat_167', 'feat_168', 'feat_169', 'feat_170', 'feat_171', 'feat_172', 'feat_173', 'feat_174', 'feat_175', 'feat_176', 'feat_177', 'feat_178', 'feat_179', 'feat_180', 'feat_181', 'feat_182', 'feat_183', 'feat_184', 'feat_185', 'feat_186', 'feat_187', 'feat_188', 'feat_189', 'feat_190', 'feat_191', 'feat_192', 'feat_193', 'feat_194', 'feat_195', 'feat_196', 'feat_197', 'feat_198', 'feat_199', 'feat_200', 'feat_201', 'feat_202', 'feat_203', 'feat_204', 'feat_205', 'feat_206', 'feat_207', 'feat_208', 'feat_209', 'feat_210', 'feat_211', 'feat_212', 'feat_213', 'feat_214', 'feat_215', 'feat_216', 'feat_217', 'feat_218', 'feat_219', 'feat_220', 'feat_221', 'feat_222', 'feat_223', 'feat_224', 'feat_225', 'feat_226', 'feat_227', 'feat_228', 'feat_229', 'feat_230', 'feat_231', 'feat_232', 'feat_233', 'feat_234', 'feat_235', 'feat_236', 'feat_237', 'feat_238', 'feat_239', 'feat_240', 'feat_241', 'feat_242', 'feat_243', 'feat_244', 'feat_245', 'feat_246', 'feat_247', 'feat_248', 'feat_249', 'feat_250', 'feat_251', 'feat_252', 'feat_253', 'feat_254', 'feat_255', 'feat_256', 'feat_257', 'feat_258', 'feat_259', 'feat_260', 'feat_261', 'feat_262', 'feat_263', 'feat_264', 'feat_265', 'feat_266', 'feat_267', 'feat_268', 'feat_269', 'feat_270', 'feat_271', 'feat_272', 'feat_273', 'feat_274', 'feat_275', 'feat_276', 'feat_277', 'feat_278', 'feat_279', 'feat_280', 'feat_281', 'feat_282', 'feat_283', 'feat_284', 'feat_285', 'feat_286', 'feat_287', 'feat_288', 'feat_289', 'feat_290', 'feat_291', 'feat_292', 'feat_293', 'feat_294', 'feat_295', 'feat_296', 'feat_297', 'feat_298', 'feat_299', 'feat_300', 'feat_301', 'feat_302', 'feat_303', 'feat_304', 'feat_305', 'feat_306', 'feat_307', 'feat_308', 'feat_309', 'feat_310', 'feat_311', 'feat_312', 'feat_313', 'feat_314', 'feat_315', 'feat_316', 'feat_317', 'feat_318', 'feat_319', 'feat_320', 'feat_321', 'feat_322', 'feat_323', 'feat_324', 'feat_325', 'feat_326', 'feat_327', 'feat_328', 'feat_329', 'feat_330', 'feat_331', 'feat_332', 'feat_333', 'feat_334', 'feat_335', 'feat_336', 'feat_337', 'feat_338', 'feat_339', 'feat_340', 'feat_341', 'feat_342', 'feat_343', 'feat_344', 'feat_345', 'feat_346', 'feat_347', 'feat_348', 'feat_349', 'feat_350', 'feat_351', 'feat_352', 'feat_353', 'feat_354', 'feat_355', 'feat_356', 'feat_357', 'feat_358', 'feat_359', 'feat_360', 'feat_361', 'feat_362', 'feat_363', 'feat_364', 'feat_365', 'feat_366', 'feat_367', 'feat_368', 'feat_369', 'feat_370', 'feat_371', 'feat_372', 'feat_373', 'feat_374', 'feat_375', 'feat_376', 'feat_377', 'feat_378', 'feat_379', 'feat_380', 'feat_381', 'feat_382', 'feat_383', 'feat_384', 'feat_385', 'feat_386', 'feat_387', 'feat_388', 'feat_389', 'feat_390', 'feat_391', 'feat_392', 'feat_393', 'feat_394', 'feat_395', 'feat_396', 'feat_397', 'feat_398', 'feat_399', 'feat_400', 'feat_401', 'feat_402', 'feat_403', 'feat_404', 'feat_405', 'feat_406', 'feat_407', 'feat_408', 'feat_409', 'feat_410', 'feat_411', 'feat_412', 'feat_413', 'feat_414', 'feat_415', 'feat_416', 'feat_417', 'feat_418', 'feat_419', 'feat_420', 'feat_421', 'feat_422', 'feat_423', 'feat_424', 'feat_425', 'feat_426', 'feat_427', 'feat_428', 'feat_429', 'feat_430', 'feat_431', 'feat_432', 'feat_433', 'feat_434', 'feat_435', 'feat_436', 'feat_437', 'feat_438', 'feat_439', 'feat_440', 'feat_441', 'feat_442', 'feat_443', 'feat_444', 'feat_445', 'feat_446', 'feat_447', 'feat_448', 'feat_449', 'feat_450', 'feat_451', 'feat_452', 'feat_453', 'feat_454', 'feat_455', 'feat_456', 'feat_457', 'feat_458', 'feat_459', 'feat_460', 'feat_461', 'feat_462', 'feat_463', 'feat_464', 'feat_465', 'feat_466', 'feat_467', 'feat_468', 'feat_469', 'feat_470', 'feat_471', 'feat_472', 'feat_473', 'feat_474', 'feat_475', 'feat_476', 'feat_477', 'feat_478', 'feat_479', 'feat_480', 'feat_481', 'feat_482', 'feat_483', 'feat_484', 'feat_485', 'feat_486', 'feat_487', 'feat_488', 'feat_489', 'feat_490', 'feat_491', 'feat_492', 'feat_493', 'feat_494', 'feat_495', 'feat_496', 'feat_497', 'feat_498', 'feat_499', 'feat_500', 'feat_501', 'feat_502', 'feat_503', 'feat_504', 'feat_505', 'feat_506', 'feat_507', 'feat_508', 'feat_509', 'feat_510', 'feat_511', 'feat_512', 'feat_513', 'feat_514', 'feat_515', 'feat_516', 'feat_517', 'feat_518', 'feat_519', 'feat_520', 'feat_521', 'feat_522', 'feat_523', 'feat_524', 'feat_525', 'feat_526', 'feat_527', 'feat_528', 'feat_529', 'feat_530', 'feat_531', 'feat_532', 'feat_533', 'feat_534', 'feat_535', 'feat_536', 'feat_537', 'feat_538', 'feat_539', 'feat_540', 'feat_541', 'feat_542', 'feat_543', 'feat_544', 'feat_545', 'feat_546', 'feat_547', 'feat_548', 'feat_549', 'feat_550', 'feat_551', 'feat_552', 'feat_553', 'feat_554', 'feat_555', 'feat_556', 'feat_557', 'feat_558', 'feat_559', 'feat_560', 'feat_561', 'feat_562', 'feat_563', 'feat_564', 'feat_565', 'feat_566', 'feat_567', 'feat_568', 'feat_569', 'feat_570', 'feat_571', 'feat_572', 'feat_573', 'feat_574', 'feat_575', 'feat_576', 'feat_577', 'feat_578', 'feat_579', 'feat_580', 'feat_581', 'feat_582', 'feat_583', 'feat_584', 'feat_585', 'feat_586', 'feat_587', 'feat_588', 'feat_589', 'feat_590', 'feat_591', 'feat_592', 'feat_593', 'feat_594', 'feat_595', 'feat_596', 'feat_597', 'feat_598', 'feat_599', 'feat_600', 'feat_601', 'feat_602', 'feat_603', 'feat_604', 'feat_605', 'feat_606', 'feat_607', 'feat_608', 'feat_609', 'feat_610', 'feat_611', 'feat_612', 'feat_613', 'feat_614', 'feat_615', 'feat_616', 'feat_617', 'feat_618', 'feat_619', 'feat_620', 'feat_621', 'feat_622', 'feat_623', 'feat_624', 'feat_625', 'feat_626', 'feat_627', 'feat_628', 'feat_629', 'feat_630', 'feat_631', 'feat_632', 'feat_633', 'feat_634', 'feat_635', 'feat_636', 'feat_637', 'feat_638', 'feat_639', 'feat_640', 'feat_641', 'feat_642', 'feat_643', 'feat_644', 'feat_645', 'feat_646', 'feat_647', 'feat_648', 'feat_649', 'feat_650', 'feat_651', 'feat_652', 'feat_653', 'feat_654', 'feat_655', 'feat_656', 'feat_657', 'feat_658', 'feat_659', 'feat_660', 'feat_661', 'feat_662', 'feat_663', 'feat_664', 'feat_665', 'feat_666', 'feat_667', 'feat_668', 'feat_669', 'feat_670', 'feat_671', 'feat_672', 'feat_673', 'feat_674', 'feat_675', 'feat_676', 'feat_677', 'feat_678', 'feat_679', 'feat_680', 'feat_681', 'feat_682', 'feat_683', 'feat_684', 'feat_685', 'feat_686', 'feat_687', 'feat_688', 'feat_689', 'feat_690', 'feat_691', 'feat_692', 'feat_693', 'feat_694', 'feat_695', 'feat_696', 'feat_697', 'feat_698', 'feat_699', 'feat_700', 'feat_701', 'feat_702', 'feat_703', 'feat_704', 'feat_705', 'feat_706', 'feat_707', 'feat_708', 'feat_709', 'feat_710', 'feat_711', 'feat_712', 'feat_713', 'feat_714', 'feat_715', 'feat_716', 'feat_717', 'feat_718', 'feat_719', 'feat_720', 'feat_721', 'feat_722', 'feat_723', 'feat_724', 'feat_725', 'feat_726', 'feat_727', 'feat_728', 'feat_729', 'feat_730', 'feat_731', 'feat_732', 'feat_733', 'feat_734', 'feat_735', 'feat_736', 'feat_737', 'feat_738', 'feat_739', 'feat_740', 'feat_741', 'feat_742', 'feat_743', 'feat_744', 'feat_745', 'feat_746', 'feat_747', 'feat_748', 'feat_749', 'feat_750', 'feat_751', 'feat_752', 'feat_753', 'feat_754', 'feat_755', 'feat_756', 'feat_757', 'feat_758', 'feat_759', 'feat_760', 'feat_761', 'feat_762', 'feat_763', 'feat_764', 'feat_765', 'feat_766', 'feat_767', 'feat_768', 'feat_769', 'feat_770', 'feat_771', 'feat_772', 'feat_773', 'feat_774', 'feat_775', 'feat_776', 'feat_777', 'feat_778', 'feat_779', 'feat_780', 'feat_781', 'feat_782', 'feat_783', 'feat_784', 'label']
        rowstring = ','.join([str(rowdict[k]) for k in columns])
        return rowstring
    return to_csv
to_csv = make_to_csv(columns=CSV_COLUMNS)

Next, we define our primary preprocessing function. Reading through the code this creates a pipeline to read data from BigQuery, use our `to_csv` function above to make a comma separated string, then write to a file in Google Cloud Storage. 

In [ ]:
import datetime

def preprocess(EVERY_N, RUNNER):
    """
    Arguments:
        -EVERY_N: Integer. Sample one out of every N rows from the full dataset.
        Larger values will yield smaller sample
        -RUNNER: "DirectRunner" or "DataflowRunner". Specfy to run the pipeline
        locally or on Google Cloud respectively. 
    Side-effects:
        -Creates and executes dataflow pipeline. 
        See https://beam.apache.org/documentation/programming-guide/#creating-a-pipeline
    """
    job_name = "preprocess-mnist" + "-" + datetime.datetime.now().strftime("%y%m%d-%H%M%S")
    print("Launching Dataflow job {} ... hang on".format(job_name))
    OUTPUT_DIR = "gs://{0}/mnist/bq/".format(BUCKET)

    #dictionary of pipeline options
    options = {
        "staging_location": os.path.join(OUTPUT_DIR, "tmp", "staging"),
        "temp_location": os.path.join(OUTPUT_DIR, "tmp"),
        "job_name": job_name,
        "project": PROJECT,
        "runner": RUNNER,
        "region": REGION
    }
  
    #instantiate PipelineOptions object using options dictionary
    opts = beam.pipeline.PipelineOptions(flags = [], **options)

    #instantantiate Pipeline object using PipelineOptions
    with beam.Pipeline(options=opts) as p:
        for phase in ["TRAIN", "TEST"]:
            query = create_query(phase, EVERY_N)
            outfile = os.path.join(OUTPUT_DIR, "{}.csv".format(phase))
            (
                p | "read_{}".format(phase) >> beam.io.Read(beam.io.BigQuerySource(query = query, use_standard_sql = True))
                  | "tocsv_{}".format(phase) >> beam.Map(to_csv)
                  | "write_{}".format(phase) >> beam.io.Write(beam.io.WriteToText(outfile))
            )
    print("Done")

In [ ]:
OUTPUT_DIR = "gs://{0}/mnist/bq".format(BUCKET)
%env OUTPUT_DIR $OUTPUT_DIR
!gsutil ls $OUTPUT_DIR

Now that we have the preprocessing pipeline function, we can execute the pipeline locally or on the cloud. To run our pipeline locally, we specify the `RUNNER` variable as `DirectRunner`. To run our pipeline in the cloud, we set `RUNNER` to be `DataflowRunner`. In either case, this variable is passed to the options dictionary that we use to instantiate the pipeline. 

As with training a model, it is good practice to test your preprocessing pipeline locally with a subset of your data before running it against your entire dataset.

### Run Beam pipeline locally

We'll start by testing our pipeline locally. This takes upto 5 minutes. You will see a message "Done" when it has finished.

In [ ]:
preprocess("50*1000", "DirectRunner")

### Run Beam pipeline on Cloud Dataflow¶

Again, we'll clear out our bucket to GCS to ensure a fresh run.

In [ ]:
%%bash
       
if gsutil ls -r $OUTPUT_DIR | grep csv; then
    gsutil -m rm -rf $OUTPUT_DIR
fi

The following step will take **15-20 minutes**. Monitor job progress on the Dataflow section of [GCP Console](https://console.cloud.google.com/dataflow). Note, you can change the first arugment to "None" to process the full dataset.

> Error, due to missing service account which is created when created a project [`[PROJECT_NUMBER]-compute@developer.gserviceaccount.com`](https://cloud.google.com/compute/docs/access/service-accounts#compute_engine_default_service_account)
>
> NEVER delete it!

Once the job finishes, we can look at the files that have been created and have a look at what they contain. You will notice that the files have been sharded into many csv files.

In [ ]:
%%bash
gsutil ls -l $OUTPUT_DIR

In [ ]:
%%bash
gsutil cat "$OUTPUT_DIR/TRAIN.csv-00000-of-*" | head